In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.utils.data import DataLoader, Dataset
import torch
import pandas as pd
from datasets import load_dataset, load_from_disk
from pathlib import Path



device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print(device)

mps


In [2]:
# student model

student_tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it")
student_model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b-it",
    torch_dtype=torch.float32
).to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
student_tokenizer.add_special_tokens({"pad_token":"<pad>"})
student_model.generation_config.pad_token_id = student_tokenizer.pad_token_id

In [ ]:
# Testing cell
inputs = student_tokenizer('The quadratic formula is', return_tensors="pt").to(device)



outputs = student_model.generate(**inputs, max_new_tokens=50, do_sample=False, truncate=True, eos_token_id=50256, ).to(device)
output_answer = student_tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
output_answer


'The quadratic formula is a very useful tool for solving problems involving the quadratic equation. It is also used to solve problems involving the cubic equation.\n\nThe quadratic formula is a very useful tool for solving problems involving the quadratic equation. It is'

In [ ]:
t = student_tokenizer('The quadratic formula is a very useful tool for solving problems involving the quadratic equation. It is also used to solve problems involving the cubic equation.\n\nThe quadratic formula is a very useful tool for solving problems involving the quadratic equation. It is also used to solve problems involving the cubic equation.\n\nThe quadratic formula is a very useful tool for solving problems involving the quadratic equation. It is also used to solve problems involving the cubic equation.\n\nThe quadratic formula is a very useful tool for solving problems involving the quadratic equation. It is also used to solve problems involving the cubic equation.\n\nThe quadratic formula is a very useful tool for solving problems involving the quadratic equation.', return_tensors="pt")
len(t['input_ids'][0].tolist())

150

In [2]:
gsm8k = load_dataset("openai/gsm8k", "main")
gsm8k_test = gsm8k['test'].to_pandas()
gsm8k_train = gsm8k['train'].to_pandas()

In [17]:
def save_df(df : pd.DataFrame, model : str, dist_folder : str, edition : str):
    base_path = Path()
    base_path = (base_path / '..' /'..'/ 'logs' / model / dist_folder / f'{edition}.csv').resolve()
    df.to_csv(base_path)

def assess_model(df, model : str, dist_folder : str, subset : str, prompt_function, tokenizer, generator, max_tokens = 50, start = 0, end = None):
    numrows = df.shape[0]
    if not end:
        end = numrows
    i = 0
    df['model_answer'] = ''
    df['model_output_tokens'] = 0
    df['model_max_tokens'] = False
    for i, row in df[start:end].iterrows():
        tokens = tokenizer(prompt_function(row['question']))
        val, out_tokens = generator(tokens, max_tokens)
        out_token_count = len(out_tokens[0].tolist()) - len(tokens['input_ids'][0].tolist())
        df.loc[i, 'model_answer'] = val
        df.loc[i, 'model_output_tokens'] = out_token_count
        df.loc[i, 'model_max_tokens'] = True if out_token_count == max_tokens else False
        i += 1
        if i % 5 == 0:
            print(f'Completed : {i} out of {numrows}')
            print(f'Completed : {round(i/numrows, 4) * 100}%')
            print('\n')
        if i % 10 == 0:
            print(f'Completed : {i} out of {numrows}')
            save_df(df, model, dist_folder, f'intermediate_{i}')
    save_df(df, model, dist_folder, f'FINAL_Completed({start} : {end})')


In [6]:
def default_prompt(question):
    return question + '\n Please respond with just the answer. The answer is:'

In [7]:
def student_tokenizerd(prompt):
    tokens = student_tokenizer(prompt, return_tensors="pt").to(device)
    return tokens

In [8]:
def student_generation(tokens, max_tokens=50):
    outputs = student_model.generate(**tokens, max_new_tokens=max_tokens, do_sample=False).to(device)
    output_answer = student_tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    return output_answer, outputs

In [ ]:
hi, to = student_generation(student_tokenizerd(default_prompt(gsm8k_test['question'][6])), 50)

In [ ]:
assess_model(gsm8k_test, 'gemma2-2b-it', 'no-distil-test', 'test', default_prompt, student_tokenizerd, student_generation, 300)

Completed : 5 out of 1319
Completed : 0.38%


Completed : 10 out of 1319
Completed : 0.76%


Completed : 10 out of 1319
Completed : 15 out of 1319
Completed : 1.1400000000000001%


Completed : 20 out of 1319
Completed : 1.52%


Completed : 20 out of 1319
Completed : 25 out of 1319
Completed : 1.9%


Completed : 30 out of 1319
Completed : 2.27%


Completed : 30 out of 1319
Completed : 35 out of 1319
Completed : 2.65%


Completed : 40 out of 1319
Completed : 3.0300000000000002%


Completed : 40 out of 1319
Completed : 45 out of 1319
Completed : 3.4099999999999997%


Completed : 50 out of 1319
Completed : 3.7900000000000005%


Completed : 50 out of 1319
Completed : 55 out of 1319
Completed : 4.17%


Completed : 60 out of 1319
Completed : 4.55%


Completed : 60 out of 1319
Completed : 65 out of 1319
Completed : 4.93%


Completed : 70 out of 1319
Completed : 5.3100000000000005%


Completed : 70 out of 1319
Completed : 75 out of 1319
Completed : 5.6899999999999995%


Completed : 80 out of 13

## Re-assess Data

In [18]:
def re_assess_model(df, model : str, dist_folder : str, subset : str, prompt_function, tokenizer, generator, vals, max_tokens = 50):
    numrows = len(vals)
    c = 0
    df['model_answer'] = ''
    df['model_output_tokens'] = 0
    df['model_max_tokens'] = False
    for i in vals:
        row = df.iloc[i]
        tokens = tokenizer(prompt_function(row['question']))
        val, out_tokens = generator(tokens, max_tokens)
        out_token_count = len(out_tokens[0].tolist()) - len(tokens['input_ids'][0].tolist())
        df.loc[i, 'model_answer'] = val
        df.loc[i, 'model_output_tokens'] = out_token_count
        df.loc[i, 'model_max_tokens'] = True if out_token_count == max_tokens else False
        
        print(c)
        if c % 3 == 0:
            print(f'Completed : {c} out of {numrows}')
            print(f'Completed : {round(c/numrows, 4) * 100}%')
            save_df(df, model, dist_folder, f'intermediate_reasses{c}')
            print('\n')
        c += 1
    save_df(df, model, dist_folder, f'FINAL_REASSESS)')

In [19]:
vals = [95, 171, 243, 429, 464, 489, 505, 593, 726, 1094, 1159, 1200]

re_assess_model(gsm8k_test, 'gemma2-2b-it', 'no-distil-test', 'test', default_prompt, student_tokenizerd, student_generation, vals, 50)

0
Completed : 0 out of 12
Completed : 0.0%


1
2
3
Completed : 3 out of 12
Completed : 25.0%


4
5
6
Completed : 6 out of 12
Completed : 50.0%


7
8
9
Completed : 9 out of 12
Completed : 75.0%


10
11


# Load data

In [ ]:
def load_disk(model : str, dist_folder : str, edition : str = 'FINAL'):
    base_path = Path()
    base_path = base_path / '..' / 'logs' / model / dist_folder / edition
    return load_from_disk(base_path)

In [ ]:
gsm8k_data = load_disk('gpt2-xl', 'no-distil-test', 'intermediate_5')

In [ ]:
gsm8k_data['test']

Dataset({
    features: ['question', 'answer'],
    num_rows: 1319
})